In [12]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

loans = pd.read_csv('./data/loan.csv')


loans.head()

/Users/eli/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exploratory Data Analysis 

In [14]:
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [24]:
features = ['id','member_id','loan_amnt','term','int_rate','grade',
            'emp_length', 'home_ownership','annual_inc','loan_status']

loans_data = loans[features]

loans_data.head()

,id,member_id,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,loan_status
0,1077501,1296599,5000.0,36 months,10.65,B,10+ years,RENT,24000.0,Fully Paid
1,1077430,1314167,2500.0,60 months,15.27,C,< 1 year,RENT,30000.0,Charged Off
2,1077175,1313524,2400.0,36 months,15.96,C,10+ years,RENT,12252.0,Fully Paid
3,1076863,1277178,10000.0,36 months,13.49,C,10+ years,RENT,49200.0,Fully Paid
4,1075358,1311748,3000.0,60 months,12.69,B,1 year,RENT,80000.0,Current


##### Observe unique values of target variable: 'loan_status'

In [38]:
print("Unique value count for 'loan_status' {}".format(loans_data['loan_status'].nunique()))
print("Unique values for 'loan_status' {}".format(loans_data['loan_status'].unique()))
loans_data['loan_status'].value_counts()

Unique value count for 'loan_status' 10
Unique values for 'loan_status' ['Fully Paid' 'Charged Off' 'Current' 'Default' 'Late (31-120 days)'
 'In Grace Period' 'Late (16-30 days)'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off' 'Issued']


Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

We will be predicting default vs fully paid, therefore we can remove instances of a different value.

In [42]:
loan_status = ['Default','Fully Paid']
loans_data = loans_data[loans_data['loan_status'].isin(loan_status)]

loans_data['loan_status'].value_counts()

Fully Paid    207723
Default         1219
Name: loan_status, dtype: int64